### Day 14: Part 2

Changes:
- bitmask bit of `0` is now ignored
- bitmask bit of `1` still overwrites
- bitmask bit of `x` is just going to mean all combinations are possible....

In [127]:
def bitmask2(mask, intVal):
    """
    Handle mask & integer, apply mask, and output integer
    Build out all possible combinations of floating bits (which can be 1 or 0)
    """
    converted_val = bin(intVal + 2**36)
    change_val = str(converted_val[3:])
    new_val = ''
    for i in range(len(mask)):
        if mask[i] == '1':
            new_val = new_val + '1'
        elif mask[i] == 'X':
            new_val = new_val + 'F' # makes it easier to find floating
        else:
            new_val = new_val + str(change_val[i])
    return new_val

In [128]:
# ex 1: 
mask = "000000000000000000000000000000X1001X"
integerVal = 100
memory_address = 42
bitmask2(mask, memory_address)

'000000000000000000000000000000F1101F'

In [129]:
# ex 2: 
mask = "00000000000000000000000000000000X0XX"
memory_address = 26
bitmask2(mask, memory_address)

'00000000000000000000000000000001F0FF'

### Now need a clean way to expand out for floating bits

In [130]:
# Very cool:
# https://stackoverflow.com/questions/4928297/all-permutations-of-a-binary-sequence-x-bits-long
["".join(seq) for seq in product("01", repeat=3)]

['000', '001', '010', '011', '100', '101', '110', '111']

In [149]:
from itertools import product
import math

def expansionFunc(inputstr):
    """Depending on number of Fs, we will have 2^TOT_F changes"""
    tot_x = inputstr.count('F')
    new_adds = ["".join(seq) for seq in product("01", repeat=int(tot_x))]
    # to do: may want to sum these outputs instead of just outputting a list
    output_list = []
    for i in range(len(new_adds)):
        new_val = ''
        idx = 0
        for j in range(len(inputstr)):
            if inputstr[j] == 'F':
                new_val = new_val + new_adds[i][idx]
                idx += 1
            else:
                new_val = new_val + inputstr[j]
        
        # append
        output_list.append(new_val)
    
    return output_list

In [150]:
# ex 1:
expansionFunc("000000000000000000000000000000F1101F")

['000000000000000000000000000000011010',
 '000000000000000000000000000000011011',
 '000000000000000000000000000000111010',
 '000000000000000000000000000000111011']

In [151]:
# ex2
expansionFunc("00000000000000000000000000000001F0FF")

['000000000000000000000000000000010000',
 '000000000000000000000000000000010001',
 '000000000000000000000000000000010010',
 '000000000000000000000000000000010011',
 '000000000000000000000000000000011000',
 '000000000000000000000000000000011001',
 '000000000000000000000000000000011010',
 '000000000000000000000000000000011011']

### Trying with test inputs first 

- just put it all together in on massive, sloppy loop

In [152]:
# path 
filepath = "day14_test_d2.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]
storage_dict = {}
dict_main_key = 0
for line in lines:
    split_vals = line.split(" = ")
    if split_vals[0] == "mask":
        dict_main_key += 1
        storage_dict[dict_main_key] = {}
        storage_dict[dict_main_key]['mask'] = split_vals[1]
    else:
        key = ''.join([x for x in split_vals[0] if x.isdigit()])
        storage_dict[dict_main_key][key] = int(split_vals[1])

In [153]:
all_mem = {}
for key in storage_dict.keys():
    for key2 in storage_dict[key]:
        if key2 == 'mask':
            continue
        memory_address = key2
        val_to_pass = storage_dict[key][key2]
        
        # get the updated address based on memory_address
        with_floating = bitmask2(storage_dict[key]['mask'], int(memory_address))
        
        # now run through & determine new keys
        key_list = expansionFunc(with_floating)
        
        # convert to integer
        for key in key_list:
            all_mem[int(key, 2)] = val_to_pass

In [154]:
tot = 0
for k, v in all_mem.items():
    tot +=  v
print(tot)

208


### Real Data....

In [155]:
# path 
filepath = "day14_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]
storage_dict = {}
dict_main_key = 0
for line in lines:
    split_vals = line.split(" = ")
    if split_vals[0] == "mask":
        dict_main_key += 1
        storage_dict[dict_main_key] = {}
        storage_dict[dict_main_key]['mask'] = split_vals[1]
    else:
        key = ''.join([x for x in split_vals[0] if x.isdigit()])
        storage_dict[dict_main_key][key] = int(split_vals[1])

In [173]:
all_mem = {}
for key in storage_dict.keys():
    for key2 in storage_dict[key]:
        if key2 == 'mask':
            continue
        memory_address = key2
        val_to_pass = storage_dict[key][memory_address]
        
        # get the updated address based on memory_address
        with_floating = bitmask2(storage_dict[key]['mask'], int(memory_address))
        
        # now run through & determine new keys
        key_list = expansionFunc(with_floating)
        
        # convert to integer
        for k in key_list:
            all_mem[int(k, 2)] = val_to_pass

In [174]:
tot = 0
for k, v in all_mem.items():
    tot +=  v
print(tot)

3706820676200
